In [1]:
import os

In [2]:
%pwd

'e:\\reviewprediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\reviewprediction'

In [29]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/divith171/reviewprediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="divith171"
os.environ["MLFLOW_TRACKING_PASSWORD"]="f17ed95907e0d5c1e4b27b30354f6ed03187f1ba"


In [30]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [31]:
from reviewprediction.constants import *
from reviewprediction.utils.common import read_yaml, create_directories, save_json

In [32]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.XGBClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri ="https://dagshub.com/divith171/reviewprediction.mlflow"
        )

        return model_evaluation_config

In [33]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from urllib.parse import urlparse
import numpy as np
import joblib
import mlflow
import mlflow.sklearn
import json



In [34]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        accuracy = accuracy_score(actual, pred)
        f1 = f1_score(actual, pred, average='weighted')  # Weighted F1-score for multi-class
        auc = roc_auc_score(actual, pred)
        return accuracy, f1, auc
    


    def log_into_mlflow(self):

        test_data = self.config.test_data_path
         # Load the JSON data
        try:
            
            with open(test_data, "r") as f:
                test_data = json.load(f)
        except FileNotFoundError:
            print("Error: JSON files not found at specified paths.")
        else:
            print("Train and test data loaded from JSON files.")
        X_test_final = []
        y_test = []
        for item in test_data:
            X_test_final.append(item["features"])
            y_test = [item["label"] for item in test_data]  # Shortened list comprehension
        X_test = np.array(X_test_final)

        model = joblib.load(self.config.model_path)
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store= urlparse(mlflow.get_tracking_uri()).scheme



        with mlflow.start_run():

          predicted_qualities = model.predict(X_test)

          (accuracy, f1, auc) = self.eval_metrics(y_test, predicted_qualities)
        
          # Saving metrics as local
          scores = {"accuracy_score": accuracy, "f1 score": f1, "roc_auc_scor": auc}
          save_json(path=Path(self.config.metric_file_name), data=scores)
          mlflow.log_params(self.config.all_params)
          mlflow.log_metric("accuracy_score",accuracy)
          mlflow.log_metric("f1 score", f1)
          mlflow.log_metric("roc_auc_scor", auc)

         

In [35]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-03-14 11:45:01,363: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-14 11:45:01,369: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-14 11:45:01,386: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-14 11:45:01,389: INFO: common: created directory at: artifacts]
[2024-03-14 11:45:01,394: INFO: common: created directory at: artifacts/model_evaluation]


Train and test data loaded from JSON files.
[2024-03-14 11:45:39,882: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
